## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
# read in input files
import geopandas as gpd
import pandas as pd

fp = "activity_locations.txt"
data = pd.read_csv(fp, sep=';')

In [2]:
# Geocode activity locations
from shapely.geometry import Point, Polygon, LineString, MultiPoint
from geopandas.tools import geocode

geo = geocode(data['addr'], provider='nominatim', user_agent='exercise_3', timeout=4)

In [3]:
# Check coordinate reference system information (re-project/define if needed)

geo.crs
geo = geo.to_crs(3879)
geo.crs

<Derived Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - nominally onshore between 24°30'E and 25°30'E but may be used in adjacent areas if a municipality chooses to use one zone over its whole extent.
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989 ensemble
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [4]:
fp2 = "shopping_centers.shp"
data2 = gpd.read_file(fp2)

In [46]:
data2

,address,id,name,addr,geometry
0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland",POINT (25504598.602 6677662.109)
1,"Salaattiasema, 14-20, Mannerheimintie, Keskust...",1,Forum,"Mannerheimintie 14-20, 00100 Helsinki, Finland",POINT (25496573.542 6672878.360)
2,"H&M, 11, Piispansilta, Matinkylä, Suur-Matinky...",2,Iso-Omena,"Piispansilta 11, 02230 Espoo, Finland",POINT (25485559.440 6671971.772)
3,"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...",3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finland",POINT (25489491.076 6678322.265)
4,"Stockmann, 3, Vantaanportinkatu, Vantaanportti...",4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finland",POINT (25497943.932 6686656.982)
5,"Silta, 5, Hermannin rantatie, Verkkosaari, Kal...",5,REDI,"Hermannin rantatie 5, 00580 Helsinki, Finland",POINT (25498837.156 6674909.983)
6,"Pasilansilta, Keski-Pasila, Pasila, Keskinen s...",6,Tripla,"Pasilansilta 11, 00520 Helsinki, Finland",POINT (25496357.756 6676158.372)


In [44]:
points11 = data2['geometry'].unary_union
print(points11)
nearest_geom_test = nearest_points(orig, points11)
print(nearest_geom_test[0], nearest_geom_test[1])

MULTIPOINT (25485559.439737607 6671971.772285322, 25489491.075777505 6678322.265374556, 25496357.756184038 6676158.372235642, 25496573.542415958 6672878.360095562, 25497943.93191118 6686656.981806412, 25498837.15607624 6674909.983273512, 25504598.601623125 6677662.108634275)
POINT (3 2) POINT (25485559.439737607 6671971.772285322)


In [50]:
from shapely.ops import nearest_points

def get_nearest_point(row, df2, point_column='geometry', value_column='geometry'):
    
    # creating multi-geometry from dataframe df2 geometries
    scnd_points = df2['geometry'].unary_union
    
    # finding nearest point for the row in main dataframe and the geometry created from second dataframe
    nearest_geom = nearest_points(row[point_column], scnd_points)
    
    # assigning nearest_data as the nearest point found in the searched row
    nearest_data = df2.loc[df2['geometry'] == nearest_geom[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

C:\ProgramData\Miniconda3\envs\geo\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [54]:
geo

,geometry,address,nearest_location,nearest_address
0,POINT (25495463.035 6674670.719),"56, Töölönkatu, Töölö, Eteläinen suurpiiri, He...",POINT (25496357.756 6676158.372),Tripla
1,POINT (25498372.175 6677525.309),"5, Koreankatu, Toukola, Keskinen suurpiiri, He...",POINT (25496357.756 6676158.372),Tripla


In [58]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:

"""By using the function introduced in nearest neighbour analysis"""
geo['nearest_location'] = geo.apply(get_nearest_point, df2 = data2, point_column = 'geometry', axis=1)
geo['nearest_address'] = geo.apply(get_nearest_point, df2 = data2, point_column = 'geometry', value_column = 'name' ,axis=1)
geo['Origin'] = ['home', 'family']
for origin, dest in zip(geo['Origin'], geo['nearest_address']):
    print("Shopping center closest to", origin, "is", dest)

Shopping center closest to home is Tripla
Shopping center closest to family is Tripla


C:\ProgramData\Miniconda3\envs\geo\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


Can you think of other application cases for the nearest neighbour analysis?

I think there are lot of applications for this but the addresses should be searched automatically to make it effective. I decided to check closest shopping malls for couple of universities.

In [126]:
fp_uni = "universities.txt"
unis = pd.read_csv(fp_uni, sep=';')

In [77]:
unis

,id,name,addr
0,0,Hanken,"Arkadiankatu 22, 00100 Helsinki, Finland"
1,1,Haaga_Helia,"Ratapihantie 13, 00520 Helsinki, Finland"
2,2,Helsingin_yliopisto_keskusta,"Yliopistonkatu 4, 00100, Finland"
3,3,Aalto,"Otakaari 1, 02150 Espoo, Finland"
4,4,Viikki,"Viikin kampus, 00790 Helsinki, Finland"
5,5,Kumpula,"Gustaf Hällströmin katu 2, 00560 Helsinki, Fin..."


In [127]:
geo_uni = geocode(unis['addr'], provider='nominatim', user_agent='exercise_3', timeout=4)

In [128]:
geo_uni.crs
geo_uni = geo_uni.to_crs(3879)


In [129]:
geo_uni['id'] = [0,1,2,3,4,5]
geo_uni = geo_uni.merge(unis, on='id')

In [130]:
geo_uni['nearest_shop'] = geo_uni.apply(get_nearest_point, df2=data2, point_column= 'geometry', value_column = 'name', axis=1)

In [131]:
geo_uni

,geometry,address,id,name,addr,nearest_shop
0,POINT (25495794.198 6673144.763),"22, Arkadiankatu, Töölö, Eteläinen suurpiiri, ...",0,Hanken,"Arkadiankatu 22, 00100 Helsinki, Finland",Forum
1,POINT (25496350.539 6676547.800),"Haaga-Helia Pasilan kampus, 13, Ratapihantie, ...",1,Haaga_Helia,"Ratapihantie 13, 00520 Helsinki, Finland",Tripla
2,POINT (25497154.366 6672982.716),"UniSport, Kluuvi, 4, Yliopistonkatu, Kaisaniem...",2,Helsingin_yliopisto_keskusta,"Yliopistonkatu 4, 00100, Finland",Forum
3,POINT (25490567.102 6674840.792),"Aalto-yliopisto Kandidaattikeskus, 1, Otakaari...",3,Aalto,"Otakaari 1, 02150 Espoo, Finland",Sello
4,POINT (25500979.281 6679334.084),"Helsingin yliopisto, Viikin kampusalue, Vierin...",4,Viikki,"Viikin kampus, 00790 Helsinki, Finland",Itis
5,POINT (25497942.152 6676901.898),"Unicafe Physicum, 2, Gustaf Hällströmin katu, ...",5,Kumpula,"Gustaf Hällströmin katu 2, 00560 Helsinki, Fin...",Tripla


In [132]:
for school, shop in zip(geo_uni['name'], geo_uni['nearest_shop']):
    print("The nearest shop for", school, "is", shop)

The nearest shop for Hanken is Forum
The nearest shop for Haaga_Helia is Tripla
The nearest shop for Helsingin_yliopisto_keskusta is Forum
The nearest shop for Aalto is Sello
The nearest shop for Viikki is Itis
The nearest shop for Kumpula is Tripla
